In which I import and analyze some high-SNR data to help me fix the peak amplitude, width, and wavelength ratios appropriately.

In [1]:
import numpy as np
from os import listdir, mkdir, path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import medfilt
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
ref_file1 = "/Users/Amanda/Desktop/PtSn/Pt_hiSNR.csv"
ref_file2 = "/Users/Amanda/Desktop/PtSn/Pt3Sn_hiSNR.csv"

ref_spec1 = np.loadtxt(ref_file1, delimiter=",", skiprows=1)
ref_spec2 = np.loadtxt(ref_file2, delimiter=",", skiprows=1)

#fig,ax=plt.subplots()
#p1,=ax.plot(ref_spec1.T[0],ref_spec1.T[1],label="Pt")
#p2,=ax.plot(ref_spec2.T[0],ref_spec2.T[1],label="Pt$_3$Sn")
#ax.legend()
#ax.set_title('Nanocrystal high-SNR spectrum')
#ax.set_xlabel('q')
#ax.set_ylabel('Intensity')

fig1,ax1=plt.subplots()
ax1.plot(ref_spec1.T[0],ref_spec1.T[1])
ax1.set_title('Pt nanocrystal high-SNR spectrum')
ax1.set_xlabel('q')
ax1.set_ylabel('Intensity')

fig2,ax2=plt.subplots()
ax2.plot(ref_spec2.T[0],ref_spec2.T[1])
ax2.set_title('Pt$_3$Sn nanocrystal high-SNR spectrum')
ax2.set_xlabel('q')
ax2.set_ylabel('Intensity')


ref_q1 = ref_spec1.T[0]
ref_q2 = ref_spec2.T[0]

bg_region1 = ((ref_q1<2.45) | ((ref_q1>3.5)&(ref_q1<4.26)) | ((ref_q1>4.78)&(ref_q1<5.04)) 
               | ((ref_q1>5.97)&(ref_q1<6.22)))
bg_region2 = ((ref_q2<2.45) | ((ref_q2>3.5)&(ref_q2<4.26)) | ((ref_q2>4.78)&(ref_q2<5.04)) 
               | ((ref_q2>5.85)&(ref_q2<6.22)))

#def poly3(x,a,b,c,d):
#    return a + b*x + c*x**2 + d*x**3

#def poly4(x,a,b,c,d,e):
#    return a + b*x + c*x**2 + d*x**3 + e*x**4

def poly5(x,a,b,c,d,e,f):
    return a + b*x + c*x**2 + d*x**3 + e*x**4 + f*x**5


def local_error_estimate_1d(intensity):
    error_estimate = np.zeros(intensity.shape, dtype=float)
    error_estimate[1:-1] = (intensity[1:-1] - intensity[2:])**2 \
                          + (intensity[1:-1] - intensity[:-2])**2
    error_estimate[0] = error_estimate[1]
    error_estimate[-1] = error_estimate[-2]
    error_estimate = (error_estimate / 4.) ** 0.5
    return error_estimate

err_est1 = local_error_estimate_1d(ref_spec1.T[1])
err_est2 = local_error_estimate_1d(ref_spec2.T[1])

popt1,pcov1 = curve_fit(poly5, ref_spec1.T[0][bg_region1], ref_spec1.T[1][bg_region1],sigma=err_est1[bg_region1])
ax1.plot(ref_q1,poly5(ref_q1,*popt1),ls=':')
ax1.plot(ref_spec1.T[0][bg_region1],np.ones(bg_region1.sum()),ls='none',marker=',',c='r')


popt2,pcov2 = curve_fit(poly5, ref_spec2.T[0][bg_region2], ref_spec2.T[1][bg_region2],sigma=err_est2[bg_region2])
ax2.plot(ref_q2,poly5(ref_q2,*popt2),ls=':')
ax2.plot(ref_spec2.T[0][bg_region2],np.ones(bg_region2.sum()),ls='none',marker=',',c='r')

sub1_q = ref_spec1.T[0]
sub1_I = ref_spec1.T[1] - poly5(sub1_q,*popt1)
sub2_q = ref_spec2.T[0]
sub2_I = ref_spec2.T[1] - poly5(sub2_q,*popt2)


In [3]:
fig1,ax1=plt.subplots()
ax1.plot(sub1_q,sub1_I)
ax1.plot(sub1_q,0*sub1_I,c='k',ls=':')
ax1.set_title('Background-subrtracted Pt nanocrystal high-SNR spectrum, PV fits')
ax1.set_xlabel('q')
ax1.set_ylabel('Intensity')

fig2,ax2=plt.subplots()
ax2.plot(sub2_q,sub2_I)
ax2.plot(sub2_q,0*sub2_I,c='k',ls=':')
ax2.set_title('Background-subrtracted Pt$_3$Sn nanocrystal high-SNR spectrum, PV fits')
ax2.set_xlabel('q')
ax2.set_ylabel('Intensity')

def gauss(x, a, x0, sigma):
    return a * np.exp(-0.5*((x - x0)/sigma)**2)

p0a = [250., 2.783, 0.04]
p0b = [10., 3., 0.04]
p0c = [100., 2.783*(4./3)**0.5, 0.04]
p0d = [50., 4.55, 0.04]
p0e = [40., 5.34, 0.04]
p0f = [10., 5.58, 0.04]
#p0g = [5., 6.43, 0.04]

boundsa = [[150., 2.683, 0.02],
           [350., 2.883, 0.08]]
boundsb = [[3., 2.9, 0.02],
           [30., 3.1, 0.08]]
boundsc = [[50., 3.1, 0.02],
           [200., 3.3, 0.08]]
boundsd = [[50., 4.45, 0.02],
           [100., 4.65, 0.08]]
boundse = [[20., 5.24, 0.02],
           [80., 5.44, 0.08]]
boundsf = [[3., 5.48, 0.02],
           [30., 5.68, 0.08]]
#boundsg = [[1., 6.4, 0.02],
#           [10., 6.5, 0.08]]

fg_region1a_g = (ref_q1>2.6) & (ref_q1<2.9)
fg_region1b_g = (ref_q1>2.9) & (ref_q1<3.1)
fg_region1c_g = (ref_q1>3.1) & (ref_q1<3.3)
fg_region1d_g = (ref_q1>4.45) & (ref_q1<4.65)
fg_region1e_g = (ref_q1>5.25) & (ref_q1<5.4)
fg_region1f_g = (ref_q1>5.5) & (ref_q1<5.65)
#fg_region1g_g = (ref_q1>6.4) & (ref_q1<6.5)

fg_region2a_g = (ref_q2>2.6) & (ref_q2<2.9)
fg_region2b_g = (ref_q2>2.9) & (ref_q2<3.1)
fg_region2c_g = (ref_q2>3.1) & (ref_q2<3.3)
fg_region2d_g = (ref_q2>4.45) & (ref_q2<4.65)
fg_region2e_g = (ref_q2>5.25) & (ref_q2<5.4)
fg_region2f_g = (ref_q2>5.5) & (ref_q2<5.65)
#fg_region2g_g = (ref_q2>6.4) & (ref_q2<6.5)


popt1a,pcov1a = curve_fit(gauss, sub1_q[fg_region1a_g], sub1_I[fg_region1a_g], p0=p0a, bounds=boundsa)
popt1b,pcov1b = curve_fit(gauss, sub1_q[fg_region1b_g], sub1_I[fg_region1b_g], p0=p0b, bounds=boundsb)
popt1c,pcov1c = curve_fit(gauss, sub1_q[fg_region1c_g], sub1_I[fg_region1c_g], p0=p0c, bounds=boundsc)
popt1d,pcov1d = curve_fit(gauss, sub1_q[fg_region1d_g], sub1_I[fg_region1d_g], p0=p0d, bounds=boundsd)
popt1e,pcov1e = curve_fit(gauss, sub1_q[fg_region1e_g], sub1_I[fg_region1e_g], p0=p0e, bounds=boundse)
popt1f,pcov1f = curve_fit(gauss, sub1_q[fg_region1f_g], sub1_I[fg_region1f_g], p0=p0f, bounds=boundsf)
#popt1g,pcov1g = curve_fit(gauss, sub1_q[fg_region1g_g], sub1_I[fg_region1g_g], p0=p0g, bounds=boundsg)

popt2a,pcov2a = curve_fit(gauss, sub2_q[fg_region2a_g], sub2_I[fg_region2a_g], p0=p0a, bounds=boundsa)
popt2b,pcov2b = curve_fit(gauss, sub2_q[fg_region2b_g], sub2_I[fg_region2b_g], p0=p0b, bounds=boundsb)
popt2c,pcov2c = curve_fit(gauss, sub2_q[fg_region2c_g], sub2_I[fg_region2c_g], p0=p0c, bounds=boundsc)
popt2d,pcov2d = curve_fit(gauss, sub2_q[fg_region2d_g], sub2_I[fg_region2d_g], p0=p0d, bounds=boundsd)
popt2e,pcov2e = curve_fit(gauss, sub2_q[fg_region2e_g], sub2_I[fg_region2e_g], p0=p0e, bounds=boundse)
popt2f,pcov2f = curve_fit(gauss, sub2_q[fg_region2f_g], sub2_I[fg_region2f_g], p0=p0f, bounds=boundsf)
#popt2g,pcov2g = curve_fit(gauss, sub2_q[fg_region2g_g], sub2_I[fg_region2g_g], p0=p0g, bounds=boundsg)


fg_region1a_pv = (ref_q1>2.35) & (ref_q1<2.9)
#fg_region1b_pv = (ref_q1>2.9) & (ref_q1<3.1)
fg_region1c_pv = (ref_q1>3.1) & (ref_q1<4.26)
fg_region1d_pv = (ref_q1>3.5) & (ref_q1<5.04)
fg_region1e_pv = (ref_q1>4.78) & (ref_q1<5.4)
#fg_region1f_pv = (ref_q1>5.5) & (ref_q1<5.65)
#fg_region1g_pv = (ref_q1>6.4) & (ref_q1<6.5)

fg_region2a_pv = (ref_q2>2.35) & (ref_q2<2.9)
#fg_region2b_pv = (ref_q2>2.9) & (ref_q2<3.1)
fg_region2c_pv = (ref_q2>3.1) & (ref_q2<4.26)
fg_region2d_pv = (ref_q2>3.5) & (ref_q2<5.04)
fg_region2e_pv = (ref_q2>4.78) & (ref_q2<5.4)
#fg_region2f_pv = (ref_q2>5.5) & (ref_q2<5.65)
#fg_region2g_pv = (ref_q2>6.4) & (ref_q2<6.5)


p0a = list([popt1a[0]*0.9,popt1a[0]*0.1,popt1a[1],popt1a[2]])
boundsa = list([[boundsa[0][0]*0.1,boundsa[0][0]*0.05,boundsa[0][1],boundsa[0][2]],
                [boundsa[1][0],boundsa[1][0]*0.9,boundsa[1][1],boundsa[1][2]]])
p0c = list([popt1c[0]*0.9,popt1c[0]*0.1,popt1c[1],popt1c[2]])
boundsc = list([[boundsc[0][0]*0.1,boundsc[0][0]*0.05,boundsc[0][1],boundsc[0][2]],
                [boundsc[1][0],boundsc[1][0]*0.9,boundsc[1][1],boundsc[1][2]]])
p0d = list([popt1d[0]*0.9,popt1d[0]*0.1,popt1d[1],popt1d[2]])
boundsd = list([[boundsd[0][0]*0.1,boundsd[0][0]*0.05,boundsd[0][1],boundsd[0][2]],
                [boundsd[1][0],boundsd[1][0]*0.9,boundsd[1][1],boundsd[1][2]]])
p0e = list([popt1e[0]*0.9,popt1e[0]*0.1,popt1e[1],popt1e[2]])
boundse = list([[boundse[0][0]*0.1,boundse[0][0]*0.05,boundse[0][1],boundse[0][2]],
                [boundse[1][0],boundse[1][0]*0.9,boundse[1][1],boundse[1][2]]])

def pseudovoigt(x, aG, aL, x0, sigma):
    x_ = (x - x0)/sigma
    return aG*np.exp(-0.5*(x_**2)) + aL*(1 + (x_**2))**-1



popt1a,pcov1a = curve_fit(pseudovoigt, sub1_q[fg_region1a_pv], sub1_I[fg_region1a_pv], p0=p0a, bounds=boundsa)
ax1.plot(sub1_q[fg_region1a_pv],pseudovoigt(sub1_q[fg_region1a_pv],*popt1a),ls='--')
popt1c,pcov1c = curve_fit(pseudovoigt, sub1_q[fg_region1c_pv], sub1_I[fg_region1c_pv], p0=p0c, bounds=boundsc)
ax1.plot(sub1_q[fg_region1c_pv],pseudovoigt(sub1_q[fg_region1c_pv],*popt1c),ls='--')
popt1d,pcov1d = curve_fit(pseudovoigt, sub1_q[fg_region1d_pv], sub1_I[fg_region1d_pv], p0=p0d, bounds=boundsd)
ax1.plot(sub1_q[fg_region1d_pv],pseudovoigt(sub1_q[fg_region1d_pv],*popt1d),ls='--')
popt1e,pcov1e = curve_fit(pseudovoigt, sub1_q[fg_region1e_pv], sub1_I[fg_region1e_pv], p0=p0e, bounds=boundse)
ax1.plot(sub1_q[fg_region1e_pv],pseudovoigt(sub1_q[fg_region1e_pv],*popt1e),ls='--')


popt2a,pcov2a = curve_fit(pseudovoigt, sub2_q[fg_region2a_pv], sub2_I[fg_region2a_pv], p0=p0a, bounds=boundsa)
ax2.plot(sub2_q[fg_region2a_pv],pseudovoigt(sub2_q[fg_region2a_pv],*popt2a),ls='--')
popt2c,pcov2c = curve_fit(pseudovoigt, sub2_q[fg_region2c_pv], sub2_I[fg_region2c_pv], p0=p0c, bounds=boundsc)
ax2.plot(sub2_q[fg_region2c_pv],pseudovoigt(sub2_q[fg_region2c_pv],*popt2c),ls='--')
popt2d,pcov2d = curve_fit(pseudovoigt, sub2_q[fg_region2d_pv], sub2_I[fg_region2d_pv], p0=p0d, bounds=boundsd)
ax2.plot(sub2_q[fg_region2d_pv],pseudovoigt(sub2_q[fg_region2d_pv],*popt2d),ls='--')
popt2e,pcov2e = curve_fit(pseudovoigt, sub2_q[fg_region2e_pv], sub2_I[fg_region2e_pv], p0=p0e, bounds=boundse)
ax2.plot(sub2_q[fg_region2e_pv],pseudovoigt(sub2_q[fg_region2e_pv],*popt2e),ls='--')

#popt2a,pcov2a = curve_fit(Pt3Sn, ref_spec2.T[0][fg_region2a], ref_spec2.T[1][fg_region2a], p0=popt1a)
#ax2.plot(ref_q2,Pt_only(ref_q2,*popt2a),ls=':')

fig1,ax1=plt.subplots()
ax1.plot(ref_spec1.T[0],ref_spec1.T[1])
ax1.plot(ref_spec1.T[0],poly5(sub1_q, *popt1)+pseudovoigt(sub1_q,*popt1a)+pseudovoigt(sub1_q,*popt1c)+pseudovoigt(sub1_q,*popt1d)+pseudovoigt(sub1_q,*popt1e),c='k',ls=':')
ax1.plot(ref_spec1.T[0][bg_region1],np.ones(bg_region1.sum()),ls='none',marker=',',c='r')
ax1.set_title('Pt nanocrystal high-SNR spectrum, PV fits')
ax1.set_xlabel('q')
ax1.set_ylabel('Intensity')

fig2,ax2=plt.subplots()
ax2.plot(ref_spec2.T[0],ref_spec2.T[1])
ax2.plot(ref_spec2.T[0],poly5(sub2_q, *popt2)+pseudovoigt(sub2_q,*popt2a)+pseudovoigt(sub2_q,*popt2c)+pseudovoigt(sub2_q,*popt2d)+pseudovoigt(sub2_q,*popt2e),c='k',ls=':')
ax2.plot(ref_spec2.T[0][bg_region2],np.ones(bg_region2.sum()),ls='none',marker=',',c='r')
ax2.set_title('Pt$_3$Sn nanocrystal high-SNR spectrum, PV fits')
ax2.set_xlabel('q')
ax2.set_ylabel('Intensity')


Some relatively excessive consistency-checking.

In [ ]:
print np.all((boundsa[1] - popt1a) > 0.001), np.all((popt1a - boundsa[0]) > 0.001)
print np.all((boundsc[1] - popt1c) > 0.001), np.all((popt1c - boundsc[0]) > 0.001)
print np.all((boundsd[1] - popt1d) > 0.001), np.all((popt1d - boundsd[0]) > 0.001)
print np.all((boundse[1] - popt1e) > 0.001), np.all((popt1e - boundse[0]) > 0.001)
print '\n'

print np.all((boundsa[1] - popt2a) > 0.001), np.all((popt2a - boundsa[0]) > 0.001)
print np.all((boundsc[1] - popt2c) > 0.001), np.all((popt2c - boundsc[0]) > 0.001)
print np.all((boundsd[1] - popt2d) > 0.001), np.all((popt2d - boundsd[0]) > 0.001)
print np.all((boundse[1] - popt2e) > 0.001), np.all((popt2e - boundse[0]) > 0.001)
print '\n\n'

print popt1a
#print popt1c
#print popt1d
#print popt1e
print '\n'

print popt2a
#print popt2c
#print popt2d
#print popt2e
print '\n\n'

def interpret(opt):
    aG, aL, x0, sigma = opt
    h = aG + aL
    I = aG*sigma*(2*np.pi)**0.5 + aL*np.pi*sigma
    #return h, I, x0, sigma
    return I, x0, sigma
    
print interpret(popt1a)
#print interpret(popt1c)
#print interpret(popt1d)
#print interpret(popt1e)
print '\n'

print interpret(popt2a)
#print interpret(popt2c)
#print interpret(popt2d)
#print interpret(popt2e)
print '\n\n'

print 'ratios of params'
print '1, c vs a', np.array(interpret(popt1c))/np.array(interpret(popt1a))
print '1, d vs a', np.array(interpret(popt1d))/np.array(interpret(popt1a))
print '1, e vs a', np.array(interpret(popt1e))/np.array(interpret(popt1a))
print '\n'
print '2, c vs a', np.array(interpret(popt2c))/np.array(interpret(popt2a))
print '2, d vs a', np.array(interpret(popt2d))/np.array(interpret(popt2a))
print '2, e vs a', np.array(interpret(popt2e))/np.array(interpret(popt2a))

def relative(a,b):
    return np.abs(a - b)/(0.5*(a + b))

print '\n\n'
print 'relative variations'
print 'c-params div by a-params', relative(np.array(interpret(popt1c))/np.array(interpret(popt1a)), np.array(interpret(popt2c))/np.array(interpret(popt2a)))
print 'd-params div by a-params', relative(np.array(interpret(popt1d))/np.array(interpret(popt1a)), np.array(interpret(popt2d))/np.array(interpret(popt2a)))
print 'e-params div by a-params', relative(np.array(interpret(popt1e))/np.array(interpret(popt1a)), np.array(interpret(popt2e))/np.array(interpret(popt2a)))


In [23]:
ref_file3 = "/Users/Amanda/Desktop/PtSn/PtSn_hiSNR.csv"
peak_ref_PtSn = "/Users/Amanda/Desktop/PtSn/PtSn_ratios.csv"

ref_spec3 = np.loadtxt(ref_file3, delimiter=",")#, usecols=(0,1))
prefits = np.loadtxt(peak_ref_PtSn, delimiter=",")#, usecols=(3,4))


indices = []
for ii in [1.961, 2.410, 3.335, 3.995, 4.391, 5.685, 5.896]:
    distance = np.abs(ref_spec3.T[0] - ii)
    nearest_index = np.where(distance == np.min(distance))[0]
    indices.append(nearest_index)
anchors = ref_spec3.T[:,indices]
anchors = anchors.reshape(2,-1)
popt3,pcov3 = curve_fit(poly5, anchors[0,:], anchors[1,:])

fig,ax=plt.subplots()
#ax.plot(ref_spec3.T[0], 8*poly5(ref_spec3.T[0],*popt1),color='b',ls='-',lw=1)
#ax.plot(ref_spec3.T[0], 6*poly5(ref_spec3.T[0],*popt2),color='g',ls='-',lw=1)
ax.plot(ref_spec3.T[0], poly5(ref_spec3.T[0],*popt3),color='b',ls='-',lw=1)
ax.plot(ref_spec3.T[0],ref_spec3.T[1],color='k',ls='-')
ax.plot(anchors[0,:], anchors[1,:],color='b',ls='none',lw=1,marker='.')
for ii in range(len(prefits.T[0])):
    distance = np.abs(ref_spec3.T[0] - prefits[ii,0])
    nearest_index = np.where(distance == np.min(distance))[0]
    ax.plot([prefits[ii,0],prefits[ii,0]],[ref_spec3.T[1][nearest_index],ref_spec3.T[1][nearest_index] - 0.2*prefits[ii,1]],color='k',ls='-')


'''def pseudovoigt(x, aG, aL, x0, sigma):
    x_ = (x - x0)/sigma
    return aG*np.exp(-0.5*(x_**2)) + aL*(1 + (x_**2))**-1
'''
ref_q3 = ref_spec3.T[0]
diff3 = ref_spec3.T[1] - poly5(ref_spec3.T[0],*popt3)

region3ia = (ref_q3<2.410)
region3ib = (ref_q3>2.410) & (ref_q3<2.95)
region3ic = (ref_q3>3.01) & (ref_q3<3.335)

p03ia = [50, 50, 2.11, 0.04]
bounds3ia = [[0, 0, 2.06, 0.02],
             [150, 150, 2.16, 0.08]]
p03ib = [160, 160, 2.89, 0.04]
bounds3ib = [[0, 0, 2.84, 0.02],
             [450, 450, 2.94, 0.08]]
p03ic = [105, 105, 3.06, 0.04]
bounds3ic = [[0, 0, 3.01, 0.02],
             [300, 300, 3.11, 0.08]]

popt3ia,pcov3ia = curve_fit(pseudovoigt, ref_q3[region3ia], diff3[region3ia],p0=p03ia,bounds=bounds3ia)
popt3ib,pcov3ib = curve_fit(pseudovoigt, ref_q3[region3ib], diff3[region3ib],p0=p03ib,bounds=bounds3ib)
popt3ic,pcov3ic = curve_fit(pseudovoigt, ref_q3[region3ic], diff3[region3ic],p0=p03ic,bounds=bounds3ic)

fig,ax=plt.subplots()
ax.plot(ref_q3, diff3, color='k',ls='-',lw=1)
ax.plot([ref_q3[0],ref_q3[-1]],[0,0],c='k',ls=':')
ax.plot(ref_q3[region3ia],pseudovoigt(ref_q3[region3ia],*popt3ia),ls='--')
ax.plot(ref_q3[region3ib],pseudovoigt(ref_q3[region3ib],*popt3ib),ls='--')
ax.plot(ref_q3[region3ic],pseudovoigt(ref_q3[region3ic],*popt3ic),ls='--')

def multi_pv(x,*pars):
    y = 0*x
    pars=list(pars)
    for ii in range(len(pars)/4):
        y += pseudovoigt(x,*pars[ii*4:ii*4+4])
    return y

def ratio_multi_pv(x,*pars):
    #aG, aL, x0, sigma
    y = 0*x
    pars=list(pars)
    for ii in range(len(pars)/4):
        locpars = copy(pars[ii*4:ii*4+4])
        if ii != 0:
            locpars[2] *= pars[2]
        y += pseudovoigt(x,*locpars)
    return y

def mix_fcc2_bcc2_etc(qq,*pars):
    pars = list(pars)
    fccpars = pars[:7]
    bccpars = pars[7:14]
    etcpars = pars[14:]
    return fcc2(qq,*fccpars) + bcc2(qq,*bccpars) + multi_pv(qq,*etcpars)

def mix_fcc2_bcc2(qq,*pars):
    pars = list(pars)
    fccpars = pars[:7]
    bccpars = pars[7:]
    return fcc2(qq,*fccpars) + bcc2(qq,*bccpars)

def fcc2(qq, aG1, aL1, x1, s1, aG2, aL2, s2):
    # Suitable for pure Pt, Pt3Sn
    return multi_pv(qq, aG1, aL1, x1, s1, aG2, aL2, x1*((4./3)**0.5), s2)

def bcc2(qq, aG1, aL1, x1, s1, aG2, aL2, s2, aG3, aL3, s3):
    # Suitable for PtSn
    return multi_pv(qq, aG1, aL1, x1, s1, aG2, aL2, x1*(0.9495**-1), s2, aG3, aL3, x1*2.11314/2.90988, s3)


'''
fcc_bcc_bounds2_I = ([0,2.69,0.025, 0,0.025, 0,2.80,0.025, 0,0.025],
                     [4,2.79,0.08,  4,0.08,  4,3.06,0.08,   4,0.08])
fcc_bcc_initialvalues2_I = [[1.027,2.74,0.05, 0.125,0.05, 0.125,3.01,0.05, 0.250,0.05],
                            [0.689,2.74,0.05, 0.062,0.05, 0.062,3.01,0.05, 0.137,0.05],
                            [0.714,2.74,0.05, 0.062,0.05, 0.062,3.01,0.05, 0.162,0.05]]

region3f = (ref_q3<3.42)
p03f = list(popt3ia)
p03f.extend(list(popt3ib))
p03f.extend(list(popt3ic))
p03f.extend([30,30,2.682,0.04])
p03f.extend([20,20,3.097,0.04])
bounds3f0 = list(bounds3ia[0])
bounds3f0.extend(bounds3ib[0])
bounds3f0.extend(bounds3ic[0])
bounds3f0.extend([0,0,2.58,0.02])
bounds3f0.extend([0,0,2.99,0.02])
bounds3f1 = list(bounds3ia[1])
bounds3f1.extend(bounds3ib[1])
bounds3f1.extend(bounds3ic[1])
bounds3f1.extend([60,60,2.83,0.08])
bounds3f1.extend([40,40,3.24,0.08])
bounds3f = [bounds3f0, bounds3f1]

popt3f,pcov3f = curve_fit(multi_pv, ref_q3[region3f], diff3[region3f],p0=p03f,bounds=bounds3f)
'''

#region3f = ((ref_q3<3.42)&(ref_q3>2.30))
region3f = (ref_q3<3.42)
p03f = list([30,30,2.682,0.04])
p03f.extend([20,20,0.04])
p03f.extend(popt3ib)
p03f.extend(np.array(popt3ic)[np.array([0,1,3])])
p03f.extend(np.array(popt3ia)[np.array([0,1,3])])
bounds3f0 = list([0,0,2.58,0.02])
bounds3f0.extend([0,0,0.02])
bounds3f0.extend(bounds3ib[0])
bounds3f0.extend(np.array(bounds3ic[0])[np.array([0,1,3])])
bounds3f0.extend(np.array(bounds3ia[0])[np.array([0,1,3])])
bounds3f1 = list([60,60,2.83,0.08])
bounds3f1.extend([40,40,0.08])
bounds3f1.extend(bounds3ib[1])
bounds3f1.extend(np.array(bounds3ic[1])[np.array([0,1,3])])
bounds3f1.extend(np.array(bounds3ia[1])[np.array([0,1,3])])
bounds3f = [bounds3f0, bounds3f1]

popt3f,pcov3f = curve_fit(mix_fcc2_bcc2, ref_q3[region3f], diff3[region3f],p0=p03f,bounds=bounds3f)

fig,ax=plt.subplots()
#for ii in range(len(popt3f)/4):
#    ax.plot(ref_q3[region3f],pseudovoigt(ref_q3[region3f],*list(popt3f)[ii*4:ii*4+4]),c='r',ls=':')
ax.plot(ref_q3[region3f],fcc2(ref_q3[region3f],*list(popt3f)[:7]),c='r',ls=':')
ax.plot(ref_q3[region3f],bcc2(ref_q3[region3f],*list(popt3f)[7:]),c='g',ls=':')


ax.plot(ref_q3, diff3, color='k',ls='-',lw=1)
ax.plot([ref_q3[0],ref_q3[-1]],[0,0],c='k',ls=':')
ax.plot(ref_q3[region3f],mix_fcc2_bcc2(ref_q3[region3f],*popt3f),c='b',ls='--')

'''
print popt3f[:4]
print bounds3ia

print popt3f[4:8]
print bounds3ib

print popt3f[8:12]
print bounds3ic
'''

print popt3f[:4]
print popt3f[4:6], popt3f[6]
print popt3f[7:11]
print popt3f[11:13], popt3f[13]
print popt3f[14:16], popt3f[16]

[ 38.56940699  13.96264926   2.76895941   0.07492511]
[  2.47912550e-13   3.30997386e-16] 0.0275207344399
[  4.20209324e+01   3.04407424e+02   2.89807139e+00   5.40852886e-02]
[ 145.50354078   21.99124344] 0.0347598821163
[ 48.62569168  47.55540061] 0.0378562375848


In [ ]:
# exception interception code from chris
import sys
try:
    pass # your fail here
except:
    print sys.exc_info()[0]
    raise

In [ ]:
help(np.loadtxt)

In [ ]:
help(str)